# RAG graph

In [1]:
%pip install transformers faiss-cpu networkx

Note: you may need to restart the kernel to use updated packages.


In [2]:
import networkx as nx
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import re


/Users/chiarapiccolo/miniconda3/envs/SA_env/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Retrieval

### Knowledge Base (KB) Graph Structure

- **Nodes**:
  - **Machine Nodes**: Represent machines with attributes like `model` and `manufacturer`.
  - **Base KPI Nodes**: Directly measured and stored in the database, such as:
    - `WorkingTime`: Measures time actively working.
    - `IdleTime`: Measures time idle but available.
    - `OfflineTime`: Measures time offline and unavailable.
  - **Non-Base KPI Nodes**: KPIs that require calculation based on Base KPIs, including:
    - `TotalAvailableTime`: Sum of `WorkingTime` and `IdleTime`.
    - `UtilizationRate`: Percentage of active working time as a function of availability.

- **Edges**:
  - **Machine-KPI Relationships**:
    - Each machine node is linked to Base KPI nodes with a `"measures"` edge.
  - **KPI Dependencies**:
    - Non-Base KPIs are connected to their required Base KPIs with `"depends_on"` edges, allowing dynamic calculation.

This organization helps the RAG agent understand:
- **Direct measurements**: From machine to Base KPIs.
- **Dependency hierarchy**: For calculated KPIs using other KPIs as input.

In [3]:
# Initialize a directed graph
G = nx.DiGraph()

# Add Machine Nodes with Attributes
G.add_node("LaserCutter_1", node_type="Machine", model="LC-200", manufacturer="Brand A")
G.add_node("LaserCutter_2", node_type="Machine", model="LC-300", manufacturer="Brand B")

# Add KPI Nodes with Descriptions and Normal Ranges
# Base KPIs (no fromula, directly saved in the DB)
G.add_node(
    "WorkingTime",
    node_type="Base KPI",
    description="Time actively working",
    unit="seconds",
    normal_min=6,
    normal_max=10)

G.add_node("IdleTime",
           node_type="Base KPI",
           description="Time idle but available",
           unit="seconds",
           normal_min=1,
           normal_max=4)

G.add_node("OfflineTime",
           node_type="Base KPI",
           description="Time offline and not available",
           unit="seconds",
           normal_min=0,
           normal_max=2)


# New KPIs (calculation needed)
# Define new KPI nodes
G.add_node(
    "TotalAvailableTime",
    node_type="NB_KPI",
    description="Total time available for work (including working and idle time)",
    unit="seconds",
    normal_min=7,
    normal_max=14,
    formula="WorkingTime + IdleTime"
)

G.add_node(
    "UtilizationRate",
    node_type="NB_KPI",
    description="Percentage of time actively working while available",
    unit="percentage",
    normal_min=60,
    normal_max=90,
    formula="(WorkingTime / (WorkingTime + IdleTime)) * 100"
)

# Add Directed Relationships (Edges) Between Machines and KPIs
G.add_edge("LaserCutter_1", "WorkingTime", relationship="measures")
G.add_edge("LaserCutter_1", "IdleTime", relationship="measures")
G.add_edge("LaserCutter_1", "OfflineTime", relationship="measures")

G.add_edge("LaserCutter_2", "WorkingTime", relationship="measures")
G.add_edge("LaserCutter_2", "IdleTime", relationship="measures")
G.add_edge("LaserCutter_2", "OfflineTime", relationship="measures")

G.add_edge("LaserCutter_1", "TotalAvailableTime", relationship ="measures")
G.add_edge("LaserCutter_1", "UtilizationRate", relationship ="measures")
G.add_edge("LaserCutter_2", "TotalAvailableTime", relationship ="measures")
G.add_edge("LaserCutter_2", "UtilizationRate", relationship ="measures")


# Add Directed Relationships (Edges) Between KPIs (base and calculated ones)
G.add_edge("TotalAvailableTime", "WorkingTime", relationship="depends_on")
G.add_edge("TotalAvailableTime", "IdleTime", relationship="depends_on")

G.add_edge("UtilizationRate", "WorkingTime", relationship="depends_on")
G.add_edge("UtilizationRate", "IdleTime", relationship="depends_on")


### Create embeddings for each node

For each node, a textual description is created.

In [4]:
# Function to generate descriptions automatically
def generate_descriptions(graph):
    descriptions = {}

    for node in graph.nodes(data=True):
        node_name, attributes = node
        node_type = attributes.get("node_type")

        if node_type == "Machine":
            # Find KPIs measured by this machine
            kpis = [
                target for source, target, data in graph.edges(data=True)
                if source == node_name and data["relationship"] == "measures"
            ]
            kpi_list = ", ".join(kpis)

            # Generate description using template for machines
            descriptions[node_name] = (
                f"it is a Machine. Model {attributes.get('model')}, "
                f"manufactured by {attributes.get('manufacturer')}. It has the following KPIs: {kpi_list}."
            )

        elif node_type == "Base KPI":
            # Find machines that measure this Base KPI
            machines = [
                source for source, target, data in graph.edges(data=True)
                if target == node_name and data["relationship"] == "measures"
            ]
            machine_list = ", ".join(machines)

            # Generate description using template for Base KPIs
            descriptions[node_name] = (
                f"it is a base KPI. It measures {attributes.get('description')} in {attributes.get('unit')}. "
                f"Reference range is {attributes.get('normal_min')} to {attributes.get('normal_max')}. "
                f"Used by machines: {machine_list}."
            )

        elif node_type == "NB_KPI":
            # Find dependencies for this Non-Base KPI
            dependencies = [
                target for source, target, data in graph.edges(data=True)
                if source == node_name and data["relationship"] == "depends_on"
            ]
            dependency_list = ", ".join(dependencies)

            # Generate description using template for Non-Base KPIs
            descriptions[node_name] = (
                f"it is a non-base KPI. It measures {attributes.get('description')} in {attributes.get('unit')}. "
                f"Reference range is {attributes.get('normal_min')} to {attributes.get('normal_max')}. "
                f"Calculated from: {dependency_list}. Formula: {attributes.get('formula')}."
            )

    return descriptions

# Generate and print descriptions for all nodes
generated_descriptions = generate_descriptions(G)
for node, desc in generated_descriptions.items():
    print(f"{node}: {desc}")

LaserCutter_1: it is a Machine. Model LC-200, manufactured by Brand A. It has the following KPIs: WorkingTime, IdleTime, OfflineTime, TotalAvailableTime, UtilizationRate.
LaserCutter_2: it is a Machine. Model LC-300, manufactured by Brand B. It has the following KPIs: WorkingTime, IdleTime, OfflineTime, TotalAvailableTime, UtilizationRate.
WorkingTime: it is a base KPI. It measures Time actively working in seconds. Reference range is 6 to 10. Used by machines: LaserCutter_1, LaserCutter_2.
IdleTime: it is a base KPI. It measures Time idle but available in seconds. Reference range is 1 to 4. Used by machines: LaserCutter_1, LaserCutter_2.
OfflineTime: it is a base KPI. It measures Time offline and not available in seconds. Reference range is 0 to 2. Used by machines: LaserCutter_1, LaserCutter_2.
TotalAvailableTime: it is a non-base KPI. It measures Total time available for work (including working and idle time) in seconds. Reference range is 7 to 14. Calculated from: WorkingTime, IdleT

Then, node descriptions are encoded into embeddings and added to the FAISS index.

In [5]:
# Load the Sentence Transformer model for embedding-based retrieval
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Assuming `generated_descriptions` contains descriptions for each node
node_embeddings = {node: embedder.encode(description) for node, description in generated_descriptions.items()}

# Convert node embeddings to a matrix for FAISS
embeddings_matrix = np.array(list(node_embeddings.values())).astype("float32")
embedding_dim = embeddings_matrix.shape[1]

# Initialize FAISS index with the correct dimension
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings_matrix)

# Map node names to their index positions in FAISS
node_to_id = {node: idx for idx, node in enumerate(node_embeddings.keys())}
id_to_node = {idx: node for node, idx in node_to_id.items()}

The following function retrieves the top-k nodes that are more similar to the user query, to retrieve the context.

In [6]:
# Function to retrieve top-k nodes using SentenceTransformer embeddings and FAISS
def retrieve_top_k_nodes(query, top_k=6):
    query_embedding = embedder.encode(query).reshape(1, -1).astype("float32")
    distances, indices = index.search(query_embedding, top_k)
    retrieved_nodes = [
        {
            "node": id_to_node[idx],
            "description": generated_descriptions[id_to_node[idx]],
            "distance": distances[0][i]
        }
        for i, idx in enumerate(indices[0])
    ]

    # Combine descriptions into a single context string
    context = "\n".join([f"{node['node']}: {node['description']}" for node in retrieved_nodes])

    return context

## Generation

The LLM model is loadeds and the context is retrieved.

### Intent detection: the first step is to determine whether the user query can be answered only by checking the KB or if it needs additional calculation and/or DB interaction.

In [7]:
# Set up the FLAN-T5 model for text generation
model_name = "google/flan-t5-large"  # or "google/flan-t5-xl"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Set up the classification pipeline
classifier_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    max_new_tokens=50,
    do_sample=False
)

# Set up text generation pipeline
generator_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    max_length=500,
    do_sample = True
)


def enough_info(query, context, pipeline = classifier_pipeline):
  prompt = f"""
    Evaluate if you can answer the following question based on the provided context: answer yes or no.

    Important: The reference ranges in the context do not represent actual historical data for any specific timeframe.

    Context:
    {context}

    Question:
    {query}

    Answer:
    """
    # Call the generator pipeline with the prompt and return the response
  response = pipeline(prompt)

    # Extract and return only the relevant part of the response
  return response[0]["generated_text"].strip()


If the query is informational (questions about definitions, descriptions, or metadata for KPIs or machines), then it can be answered.

If the query is action-oriented (request for specific KPI values, calculations or historical data), then valuable data for the KPI calculation engine are identified.

In [8]:

def informational_query(query, context, pipeline = generator_pipeline):
    # Format the prompt with the context and query for the FLAN-T5 model
    prompt = f"""
    Please answer the following question using the information provided in the "Additional Information" section.
    Your answer should be direct and concise, focusing specifically on addressing the question.

    - If the question asks for additional details, provide only the specific information requested.
    - Do not introduce information or explanations beyond what is directly asked for in the question.

    Additional Information:
    {context}

    Question:
    {query}

    Answer:
    """

    # Generate the response
    response = generator_pipeline(prompt)
    return response


def action_query(query, context, pipeline = generator_pipeline):
  prompt = f"""
    Analyze the following query and context. Identify the KPI name, machine name, and temporal range requested.

    Examples:
    1. Query: "Calculate the utilization rate for LaserCutter_2 over the past month."
       Answer:
       - Machine: "LaserCutter_2"
       - KPI: UtilizationRate
       - Period of time: past month.

    2. Query: "What was the working time for LaserCutter_1 over the past week?"
       Answer:
       - Machine: "LaserCutter_1"
       - KPI: WorkingTime
       - Period of time: past week

    Context:
    {context}

    Query:
    {query}

    """


  response = pipeline(prompt)

  return response

In [10]:
def steps(query, context):
  pattern = r'^(yes|YeS|yEs|YEs|Yes|yES|YES|yeS)$'

  step1 = enough_info(query, context)
  print(step1)

  if re.match(pattern, step1):
    print("The query can be answered")
    response = informational_query(query, context)
  else:
    print("Not enough info to answer the query")
    response = action_query(query, context)
  return response[0]["generated_text"].strip()



# Example usage
query1 = "How many Laser Cutter machines are there?"
query2 = "What is the utilization rate for LaserCutter_1 last month?"
query3 = "Show me the available KPIs for the Laser Cutter machine."
query4 = "What was the working time for LaserCutter_1 over the past week?"
query5 = "Is there a KPI for that measure the percentage of time working?"
query6 = "Is there a KPI for material cost?"


for query in [query1, query2, query3, query4, query5, query6]:
    print(f"\nQuery: {query}")
    context = retrieve_top_k_nodes(query)
    print(f"\nContext: {context}")
    response = steps(query, context)
    print("\n")
    print(response)
    print("\n")
    print("\n")


Query: How many Laser Cutter machines are there?

Context: WorkingTime: it is a base KPI. It measures Time actively working in seconds. Reference range is 6 to 10. Used by machines: LaserCutter_1, LaserCutter_2.
IdleTime: it is a base KPI. It measures Time idle but available in seconds. Reference range is 1 to 4. Used by machines: LaserCutter_1, LaserCutter_2.
OfflineTime: it is a base KPI. It measures Time offline and not available in seconds. Reference range is 0 to 2. Used by machines: LaserCutter_1, LaserCutter_2.
LaserCutter_2: it is a Machine. Model LC-300, manufactured by Brand B. It has the following KPIs: WorkingTime, IdleTime, OfflineTime, TotalAvailableTime, UtilizationRate.
LaserCutter_1: it is a Machine. Model LC-200, manufactured by Brand A. It has the following KPIs: WorkingTime, IdleTime, OfflineTime, TotalAvailableTime, UtilizationRate.
TotalAvailableTime: it is a non-base KPI. It measures Total time available for work (including working and idle time) in seconds. Ref